#UTS Machine Learning - Klasifikasi Wine
nama = Divan Gusti Irawan Prodi = Staterkom nim = 4112322021



Pada tahap awal ini, kita mengimpor library yang dibutuhkan dalam proses machine learning.

pandas dan numpy digunakan untuk memproses dan mengelola data.

train_test_split dan cross_val_score dari sklearn.model_selection digunakan untuk membagi dan mengevaluasi data.

RandomForestClassifier adalah model klasifikasi berbasis ensemble.

StandardScaler digunakan untuk normalisasi data.

classification_report dan confusion_matrix untuk mengevaluasi performa model.

GridSearchCV untuk mencari hyperparameter terbaik.

In [11]:
# 1. Import library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV


Dataset training dan testing dibaca menggunakan pandas.read_csv(). Dataset training memiliki label quality, sedangkan dataset testing tidak punya label (digunakan untuk prediksi akhir).

In [12]:
# 2. Load dataset
train = pd.read_csv("/content/data_training.csv")
test = pd.read_csv("/content/data_testing (1).csv")


Langkah ini digunakan untuk melihat ukuran dataset dan mengecek apakah ada data kosong (missing values).
Hasil pengecekan menunjukkan tidak ada data yang hilang, artinya dataset bersih dan siap diolah.
Ini penting karena missing value dapat menyebabkan error atau menurunkan akurasi model.

In [13]:
# 3. Cek data
print("Training data shape:", train.shape)
print("Testing data shape:", test.shape)
print(train.isnull().sum())

Training data shape: (857, 13)
Testing data shape: (286, 12)
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
Id                      0
dtype: int64


Data training dibagi lagi menjadi training dan validation (80:20) menggunakan train_test_split. Tujuannya agar bisa mengevaluasi performa model secara obyektif sebelum digunakan pada data testing.



In [14]:
# 4. Pisahkan fitur dan label
X = train.drop(columns=["quality", "Id"])
y = train["quality"]
X_test_final = test.drop(columns=["Id"])


Model Random Forest Classifier dilatih dengan GridSearchCV untuk mencari kombinasi hyperparameter terbaik (n_estimators, max_depth).
Model divalidasi dengan 5-fold cross-validation untuk menghindari overfitting.

In [15]:
# 5. Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test_final)

Hasil model diuji pada data validasi dan dievaluasi dengan:

Classification Report: precision, recall, f1-score.

Accuracy Validasi: ~60%

Rata-rata skor cross-validation: ~64.76%

In [16]:
# 6. Split data untuk validasi
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


Model terbaik digunakan untuk memprediksi nilai quality pada data data_testing.csv. Hasil prediksi disimpan sebagai dataframe baru.

In [17]:
# 7. Model dengan GridSearch untuk hindari overfitting
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, None], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='accuracy')

GridSearchCV untuk optimasi model

In [18]:
# 8. Evaluasi di data validasi
y_pred_val = grid.best_estimator_.predict(X_val)
print("Classification Report (Validation):")
print(classification_report(y_val, y_pred_val))


Classification Report (Validation):
              precision    recall  f1-score   support

           4       0.00      0.00      0.00         3
           5       0.64      0.66      0.65        67
           6       0.59      0.67      0.63        78
           7       0.53      0.38      0.44        21
           8       0.00      0.00      0.00         3

    accuracy                           0.60       172
   macro avg       0.35      0.34      0.34       172
weighted avg       0.58      0.60      0.59       172



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
# 9. Evaluasi Cross-Validation (skor rata-rata)
cv_scores = cross_val_score(grid.best_estimator_, X_scaled, y, cv=5)
print("Cross-validation scores:", cv_scores)
print("Average CV score:", np.mean(cv_scores))


Cross-validation scores: [0.65697674 0.6627907  0.67836257 0.67836257 0.56140351]
Average CV score: 0.647579219366245


Hasil prediksi disimpan ke file submission.csv dengan dua kolom:

Id (index dari data)

(Quality)

In [20]:
# 10. Prediksi data testing akhir
y_test_pred = grid.best_estimator_.predict(X_test_scaled)
output = pd.DataFrame({"Id": test["Id"], "Quality": y_test_pred})
output.to_csv("submission.csv", index=False)
print("Prediksi disimpan ke submission.csv")


Prediksi disimpan ke submission.csv
